# Probe Designer


## Environment


In [4]:
# basci env
import os
import pandas as pd
import time
import json
from tqdm import tqdm

# data process of file from ncbi
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqUtils import MeltingTemp as mt

# # get gene data from ncbi
# from Bio import Entrez

# # blast and xml file process
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

# add package to sys var
# os.chdir(os.path.dirname(os.path.abspath(__file__)))
# sys.path.append("../lib")

# dir
workdir = './dataset/2024.1.23_Sindy_marker_genes/'
os.makedirs(workdir, exist_ok=True)

current_time = time.localtime()
formatted_time = time.strftime("%Y%m%d_%H%M%S", current_time)

output = os.path.join(workdir, 'results', formatted_time)
pre_binding_dir = os.path.join(output, "pre_binding")
os.makedirs(output, exist_ok=True)

# basic variables
gene_name_list_tosearch = "gene_name_list_tosearch.txt"
pre_binding_file_suffix = "_pre_binding.fasta"
total_pre_binding_file_name = "_total.fasta"

# tmp file
pre_binding_num_file = "pre_binding_num.json"
blast_results_file = "blast_results.xml"

In [5]:
organism = 'mouse'
gene_info = pd.read_excel(os.path.join(workdir, "cell_type_marker_list.xlsx"), sheet_name='cell_type_list')
gene_list = list(gene_info['gene'].unique())

## Get seq from ensembl dataset

In [ ]:
# from lib.database_interaction import ensembl_id_to_seqs
# import time

# skip = 0
# trial = 0

# tmp_isoform_list = isoform_list[skip:]
# tmp_id_list = id_list[len(sequences_of_all)+skip:]

# for i in range(len(tmp_isoform_list)):
#     isoform = tmp_isoform_list[i]
#     id = tmp_id_list[i]
#     sequences_of_all[f'{id}_{isoform}'] = dict()
#     sequences = ensembl_id_to_seqs(gene=isoform, gene_id=id.split('.')[0], seq_type='cds')
#     for desc, sequence in sequences.items():
#         sequences_of_all[f'{id}_{isoform}'][desc] = sequence

In [12]:
from lib.database_interaction import ensembl_name_to_seqs
import time


sequences_of_all = dict()
skip, trial = 0, 0

while True:
    if skip == len(gene_list): break
    if trial > 3:
        skip += 1
        trial = 0
    try:
        tmp_gene_list = gene_list[skip:]
        for gene in tmp_gene_list:
            sequences_of_all[gene] = dict()
            sequences = ensembl_name_to_seqs(gene=gene, species='mouse', seq_type='cds')
            for desc, sequence in sequences.items():
                sequences_of_all[gene][desc] = sequence
            skip += 1
    except: 
        trial += 1
        time.sleep(5)
        
with open(os.path.join(output, 'sequence_of_all.json'), 'w') as file: json.dump(sequences_of_all, file)

Gene:	Nwd1: 100%|██████████| 10/10 [00:05<00:00,  1.75it/s]


In [15]:
from pprint import pprint
pprint(sequences_of_all, sort_dicts=False)

{'Ttr': {'ENSMUST00000075312|Ttr-201|protein_coding': 'ATGGCTTCCCTTCGACTCTTCCTCCTTTGCCTCGCTGGACTGGTATTTGTGTCTGAAGCTGGCCCCGCGGGTGCTGGAGAATCCAAATGTCCTCTGATGGTCAAAGTCCTGGATGCTGTCCGAGGCAGCCCTGCTGTAGACGTGGCTGTAAAAGTGTTCAAAAAGACCTCTGAGGGATCCTGGGAGCCCTTTGCCTCTGGGAAGACCGCGGAGTCTGGAGAGCTGCACGGGCTCACCACAGATGAGAAGTTTGTAGAAGGAGTGTACAGAGTAGAACTGGACACCAAATCGTACTGGAAGACACTTGGCATTTCCCCGTTCCATGAATTCGCGGATGTGGTTTTCACAGCCAACGACTCTGGCCATCGCCACTACACCATCGCAGCCCTGCTCAGCCCATACTCCTACAGCACCACGGCTGTCGTCAGCAACCCCCAGAATTGA'},
 'Tmem72': {'ENSMUST00000056623|Tmem72-201|protein_coding': 'ATGAAGCTCCAGGTATTCTGGACTGGACTGGAGTATACCTGCCGGCTCTTGGGCATCGCTACGGCTGCAGTGTTGATTGGAGTGGGCACCGAGACCTTCCTCCGGGGGCGGTTCAAAAGCCTGGCCTTCTATCTGCTGTTTACAGGAGTCACCATCTCTGTGTGTGAAGGGACCTACTTTGTGGCTCAACTCTTGGCCATCTGCTTCAAGTGCCAGCCGGGGTCTCTGGCACACAGAGCGAAGGAGAGGGCCCACTGGCTGGGCTGCTTCCAGAAGTTCCTCGCCTACATGCTGCTGTCAGTGGCCTGCTTCCTCCACCCTGTCCTGGTCTGGCATGTGACCATTCCAGGCTCCATGTTAATCATCACTGGCCTGGCCTACTTCCTGCTGAGCAAGCGAAAGAAAAAAAAGGCTGCTCCAGAGGTGGCACCCCCCA

In [17]:
longest_isoforms = {}

for gene, isoforms in sequences_of_all.items():
    longest_isoform = None
    max_length = 0
    for isoform, sequence in isoforms.items():
        if len(sequence) > max_length:
            max_length = len(sequence)
            longest_isoform = isoform
    if longest_isoform:
        longest_isoforms[longest_isoform] = isoforms[longest_isoform]

with open(os.path.join(output, 'longest_isoforms.json'), 'w') as file: json.dump(longest_isoforms, file)

## Binding site Searcher


In [19]:
from lib.search_binding import step_by_step, find_max_min_difference_fixed_length_subsequence, seq_minus

# Initiation of array
binding_site_FOIs = [
    "accession",
    "gene_name",
    "mol_type",
    "organism",
    "pos_on_seq",
    "binding",
    "Tm_l",
    "Tm_r",
    "wanted",
]
align_FOIs = ["align_num", "align_accession", "align_descrip", "plus/minus"]
FOI = pd.DataFrame(columns=binding_site_FOIs + align_FOIs)

# Search binding sites on mRNA sequence
file_out_dir = pre_binding_dir
try:
    os.mkdir(file_out_dir)
except:
    pass

pre_binding_num = {}

# initialization of file
with open(os.path.join(file_out_dir, total_pre_binding_file_name), "w") as handle:
    handle.write("")

for desc, seq in longest_isoforms.items():
    id, gene_name, mol_type = desc.split('|')
    minus_seq = seq_minus(seq)
    
    Tm_l, Tm_r, selected_substrings, pos_on_seq = step_by_step(
        minus_seq, gene=gene_name,
        BDS_len=40, BDS_num=50, min_gap=1, better_gap=40,
        G_min=0.25, G_max=0.7, G_consecutive=5, Tm_low=50, Tm_high=65,)
    
    record_list = []
    for i, pre_binding_tmp in enumerate(selected_substrings):
        record_list.append(
            SeqRecord(
                Seq(pre_binding_tmp),
                id="pre_binding" + str(i),
                description="|".join([id, gene_name, organism, mol_type]),
            )
        )

    # add information about binding sites to FOI
    add = pd.DataFrame(
        {
            "accession": [id] * len(selected_substrings),
            "gene_name": [gene_name] * len(selected_substrings),
            "mol_type": [mol_type] * len(selected_substrings),
            "organism": [organism] * len(selected_substrings),
            "binding": selected_substrings,
            "Tm_l": Tm_l,
            "Tm_r": Tm_r,
            "pos_on_seq": pos_on_seq,
        }
    )
    FOI = pd.concat([FOI, add], ignore_index=True)

    file_out = os.path.join(file_out_dir, gene_name + pre_binding_file_suffix)
    
    # write pre_binding to files
    with open(file_out, "w") as f:
        for new_record in record_list:
            SeqIO.write(new_record, f, "fasta")
    with open(file_out_dir + total_pre_binding_file_name, "a") as handle:
        for new_record in record_list:
            SeqIO.write(new_record, handle, "fasta")

    # record the num of pre_binding for each gene
    pre_binding_num[f"{id}_{gene_name}"] = len(selected_substrings)

with open(os.path.join(output, pre_binding_num_file), "w") as f:
    json.dump(pre_binding_num, f)

position_searching_Ttr-201: 100%|██████████| 316/316 [00:00<00:00, 31685.39it/s]
C:\Users\Mingchuan\AppData\Local\Temp\ipykernel_12128\2835666293.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  FOI = pd.concat([FOI, add], ignore_index=True)


Gene Ttr-201: 	Not enough pos for 50 binding sites.
Gene Ttr-201: 	condition too harsh, loose to get better results
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 30, 82, 152, 153, 154, 155, 156, 157, 158, 159, 160, 162, 165, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 189, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 267, 268, 311, 312, 313, 314, 315]


position_searching_Tmem72-201: 100%|██████████| 624/624 [00:00<00:00, 23173.77it/s]


Gene Tmem72-201: 	condition too harsh, loose to get better results
[9, 23, 77, 90, 95, 100, 110, 115, 120, 125, 130, 135, 140, 145, 150, 157, 177, 195, 237, 242, 248, 253, 258, 263, 268, 273, 326, 340, 345, 368, 388, 393, 398, 403, 408, 413, 418, 423, 428, 434, 441, 446, 451, 476, 497, 502, 507, 513, 521, 526, 541, 607, 617, 622]


position_searching_Rgs22-203: 100%|██████████| 2983/2983 [00:00<00:00, 51568.11it/s]


...

position_searching_Tmem132b-201: 100%|██████████| 2551/2551 [00:00<00:00, 26924.32it/s]


Gene Tmem132b-201: 	condition too harsh, loose to get better results
[0, 58, 88, 130, 171, 223, 253, 283, 315, 345, 375, 433, 463, 495, 603, 646, 676, 798, 828, 858, 888, 918, 961, 991, 1094, 1124, 1179, 1247, 1278, 1338, 1368, 1398, 1471, 1548, 1617, 1699, 1729, 1777, 1808, 1838, 1944, 1983, 2020, 2053, 2083, 2113, 2143, 2201, 2249, 2279, 2467, 2524]


position_searching_Cables1-202: 100%|██████████| 1389/1389 [00:00<00:00, 28422.57it/s]


Gene Cables1-202: 	condition too harsh, loose to get better results
[2, 12, 22, 38, 71, 91, 108, 221, 287, 335, 365, 388, 409, 419, 430, 440, 457, 479, 571, 622, 632, 771, 782, 792, 808, 834, 854, 864, 874, 898, 918, 928, 938, 948, 958, 972, 992, 1035, 1045, 1055, 1120, 1130, 1140, 1150, 1160, 1205, 1215, 1273, 1283, 1367]


position_searching_Nwd1-201: 100%|██████████| 3714/3714 [00:00<00:00, 28426.88it/s]


## Blast and extract blast results

In [ ]:
# with open(file_out_dir + total_pre_binding_file_name, "r") as f:
#     fasta_string = f.read()
# txid = [2697049]  # organism

# # Submit BLAST search and get handle object
# handle = NCBIWWW.qblast(
#     program="blastn",
#     megablast="yes",
#     database="refseq_rna",
#     sequence=fasta_string,
#     url_base="https://blast.ncbi.nlm.nih.gov/Blast.cgi",
#     format_object="Alignment",
#     format_type="Xml",
# )

# # read handle object and save to a file
# with open(os.path.join(os.path.join(output, blast_results_file)), "w") as f:
#     f.write(handle.read())

In [20]:
# Extract interested information from blast_results
from Bio.Blast import NCBIXML


align_num = []
# read the id/plus-minus part/align_num
with open(os.path.join(output, blast_results_file), "r") as blast_output:
    blast_records = NCBIXML.parse(blast_output)
    loca = 0
    for blast_record in blast_records:
        align_accession = []
        align_descrip_list = []
        # get align num of each binding site
        length = len(blast_record.alignments)
        align_num.append(length)
        for i in range(length):
            descrip = blast_record.descriptions[i].title.split("|")
            # get accession and descrip of each align seq
            align_accession.append(descrip[3])
            align_descrip_list.append(descrip[-1])
        FOI.loc[loca, "align_accession"] = "|".join(str(_) for _ in align_accession)

        # add align_descrip to df
        FOI.loc[loca, "align_descrip"] = "|".join(str(_) for _ in align_descrip_list)

        # get plus/minus of each align seq
        p_m = [blast_record.alignments[_].hsps[0].frame[1] for _ in range(length)]

        # add plus/minus to df
        try:
            FOI.loc[loca, "plus/minus"] = ",".join([str(_) for _ in p_m])
        except:
            FOI.loc[loca, "plus/minus"] = "NAN"

        loca += 1

FOI["align_num"] = align_num

## Select wanted binding site


In [21]:
FOI["wanted"] = [True] * len(FOI)

In [22]:
# sieve for the suitable binding site
gene_name_list = [_.upper() for _ in gene_list]
gene_name_list_out = [i for i in gene_name_list]
for i in range(len(FOI)):
    # check gene_name
    gene_name = FOI.loc[i, "gene_name"]
    if gene_name.split('-')[0].upper() not in gene_name_list:
        FOI.loc[i, "wanted"] = False
    else:
        try:
            gene_name_list_out.remove(gene_name)
        except:
            pass

    # check DNA or mRNA type
    if FOI.loc[i, "wanted"] == True:
        if FOI.loc[i, "mol_type"] != "protein_coding":
            FOI.loc[i, "wanted"] = False
            print(FOI.loc[i, "mol_type"])

    # check gene_organism name
    if FOI.loc[i, "wanted"] == True:
        spe_ori, gene_ori = FOI.loc[i, "organism"], FOI.loc[i, "gene_name"].split('-')[0]
        descrip = FOI.loc[i, "align_descrip"].split("|")
        for des in descrip:
            if gene_ori not in des and spe_ori in des:
                FOI.loc[i, "wanted"] = False
                break

    # check plus/minus
    if FOI.loc[i, "wanted"] == True:
        if pd.isnull(FOI.loc[i, "plus/minus"]):
            FOI.loc[i, "wanted"] = False
        else:
            pm_list = FOI.loc[i, "plus/minus"].split(",")
            if "-1" not in pm_list:
                FOI.loc[i, "wanted"] = False

# write the whole information of interest to a excel file in tmp dir
FOI.to_excel(os.path.join(output, "probes_sieve.xlsx"))

out_tmp = FOI[FOI["wanted"] == True]
output_df = pd.DataFrame()
for gene in out_tmp.gene_name.unique():
    pos_of_True = list(out_tmp[out_tmp.gene_name == gene]["pos_on_seq"])
    best_pos = find_max_min_difference_fixed_length_subsequence(
        pos_of_True,
        length=3,
        min_gap=40,
        better_gap=80,
        gene=gene,
    )
    out_subset = out_tmp[out_tmp.gene_name == gene]
    out_subset = out_subset[out_subset["pos_on_seq"].isin(best_pos)]
    output_df = pd.concat([output_df, out_subset])

# write the output to a xlsx file
output_df.to_excel(os.path.join(output, "probes_wanted.xlsx"))